# Extração e Transformação dos Dados

## Bibliotecas Utilizadas

In [ ]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()
API_TOKEN = str(os.getenv('API_TOKEN'))

## Extração dos dados do Million Playlist Dataset

In [ ]:
def read_json_file(file_path: str=None, url: str=None) -> dict:
    """
    Essa função lê um arquivo json de um caminho local ou de uma url
    e retorna um dicionário com o conteúdo do arquivo.

    :param file_path: caminho para o arquivo json local.
    :param url: caminho para o arquivo json online.

    :retorna: Um dicionário com o conteúdo do arquivo json.
    """

    if url is not None:
        response = requests.get(url)
        data = response.json()
        return data

    elif file_path is not None:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            return data
        

def extract_content_from_json(data: dict) -> pd.DataFrame:
    """
    Essa função extrai o conteúdo de um arquivo json e retorna um dataframe.

    :param data: Um dicionário com o conteúdo do arquivo json.

    :retorna: Um dataframe com o conteúdo do arquivo json.
    """
    df_musics = pd.DataFrame()

    # Itera sobre as playlists
    for playlist in data['playlists']:
        name_playlist = playlist['name']
        id_playlist = playlist['pid']

        print(f'{id_playlist} - Playlist: {name_playlist}')

        # Itera sobre as músicas da playlist
        musics_playlist = playlist['tracks']

        for music in musics_playlist:
            # name_musica = music['track_name']
            # id_musica = music['track_uri']

            # print(f'\tSong: {name_song} - ID: {song_id}')
            df_musica_aux = pd.DataFrame.from_dict(music, orient='index')
            df_musica_aux = df_musica_aux.T

            # Adiciona o nome da playlist e o ID da playlist
            df_musica_aux['playlist_name'] = name_playlist
            df_musica_aux['playlist_id'] = id_playlist

            df_musica_aux = df_musica_aux.T
            df_musics = pd.concat([df_musics, df_musica_aux], axis=1)

    return df_musics


In [ ]:
# Para cada arquivo de playlist, extrai o conteúdo e salva em um arquivo csv

files_used = [
    'mpd.slice.0-999.json',
    'mpd.slice.1000-1999.json',
    'mpd.slice.2000-2999.json',
    'mpd.slice.3000-3999.json',
    'mpd.slice.4000-4999.json',
    'mpd.slice.5000-5999.json',
]

In [ ]:
for file in files_used:
    version = file.split('.')[2].replace('-', '_')

    data = read_json_file(file_path=f'../data/00 - Raw Data/{file}')
    df_musics = extract_content_from_json(data)
    df_musics.to_csv(f'../data/01 - Extracted Data/df_musics_{version}.csv', index=False)


## Transformação

### Concatenar os dados extraídos


In [ ]:
df_musics_final = pd.DataFrame()

for file in files_used:
    version = file.split('.')[2].replace('-', '_')

    df = pd.read_csv(f'../data/01 - Extracted Data/df_musics_{version}.csv')

    # Mantém apenas as colunas que precisamos
    df = df[['track_uri', 'artist_uri', 'album_uri']]

    # Concatena os dataframes
    df_musics_final = pd.concat([df_musics_final, df], axis=0)

# Remover duplicatas
df_musics_final.drop_duplicates(inplace=True)

# Salva o dataframe final
df_musics_final.to_csv('../data/01 - Extracted Data/df_musics_concatenate.csv', index=False)

df_musics_final.head(5)


## Limpeza dos dados

In [ ]:
df_musics_final['track_uri'] = df_musics_final['track_uri'].str.replace('spotify:track:', '')
df_musics_final['artist_uri'] = df_musics_final['artist_uri'].str.replace('spotify:artist:', '')
df_musics_final['album_uri'] = df_musics_final['album_uri'].str.replace('spotify:album:', '')

df_musics_final.head(5)

## Funções para extrair dados da API do Spotify

In [ ]:
def make_req(route: str, id: str) -> requests.models.Response:
    """
    Essa função faz uma requisição para a API do Spotify.

    :param route: Rota da API que será acessada.
    :param id: ID do recurso que será acessado.

    :retorna: Um objeto do tipo Response.    
    """
    url = f'https://api.spotify.com/v1/{route}/{id}'

    headers = {
        'Authorization': f'Bearer {API_TOKEN}'
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f'Erro: {response.status_code}')
        print(f'Erro: {response.text}')

    else:
        return response


def get_track(id_track: str) -> pd.DataFrame:
    """
    Essa função faz uma requisição para a API do Spotify, na rota de tracks,
    e retorna um dataframe com os dados da música.

    :param id_track: ID da música que será acessada.

    :retorna: Um dataframe com os dados da música.    
    """
    data = make_req('tracks', id_track).json()
    df = pd.DataFrame(data['tracks'])
    return df


def get_audio_features(id_track: str) -> dict:
    """
    Essa função faz uma requisição para a API do Spotify, na rota de audio_features,
    e retorna um dataframe com os dados da música.

    :param id_track: ID da música que será acessada.

    :retorna: Um dataframe com as features da música.    
    """
    data = make_req('audio-features', id_track).json()
    df = pd.DataFrame(data['audio_features'])
    return df


def get_album(id_album: str) -> dict:
    """
    Essa função faz uma requisição para a API do Spotify, na rota de albums,
    e retorna um dataframe com os dados do álbum.

    :param id_album: ID do álbum que será acessado.

    :retorna: Um dataframe com os dados do álbum.    
    """
    data = make_req('albums', id_album).json()
    df = pd.DataFrame(data['albums'])
    return df


def get_artist_data(id_artist: str) -> dict:
    """
    Essa função faz uma requisição para a API do Spotify, na rota de artists,
    e retorna um dataframe com os dados do artista.

    :param id_artist: ID do artista que será acessado.

    :retorna: Um dataframe com os dados do artista.    
    """
    data = make_req('artists', id_artist).json()
    df = pd.DataFrame(data['artists'])
    return df

In [ ]:
def get_music_features(tracks_ids: list) -> pd.DataFrame:
    """
    Dado uma lista de IDs de músicas, coleta as features de cada uma delas, dos artistas e dos álbuns.

    :param tracks_ids: Uma lista com os IDs das músicas.

    :retorna: Um dataframe com as features das músicas, dos artistas e dos álbuns estruturados em linhas.
    """

    tracks_ids_str = ','.join(tracks_ids)
    tracks_ids_str = '?ids=' + tracks_ids_str

    # 1. Pega os dados da música
    tracks_list_df = get_track(tracks_ids_str)

    # Coleta o ID do album
    tracks_list_df['id_album'] = tracks_list_df['album'].apply(lambda x: x['id'])

    # Coleta o ID dos artistas
    tracks_list_df['id_artist'] = tracks_list_df['artists'].apply(lambda x: [d['id'] for d in x])

    # Expande os IDs dos artistas para linhas separadas
    tracks_list_df = tracks_list_df.explode('id_artist')

    
    # 2. Pega as features da música
    df_audio_features = pd.DataFrame()
    for i in range(0, len(tracks_ids), 50):
        tracks_ids_str = ','.join(tracks_ids[i:i+50] )
        tracks_ids_str = '?ids=' + tracks_ids_str
        df_audio_features_aux = get_audio_features(tracks_ids_str)
        df_audio_features = pd.concat([df_audio_features, df_audio_features_aux], axis=0)

    # Filtra as colunas
    df_audio_features = df_audio_features[['id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
    
    # Junta os dados das músicas com as features
    tracks_list_df = tracks_list_df.merge(df_audio_features, on='id', how='left')


    # 3. Pega os dados do album
    df_albums = pd.DataFrame()
    albums_ids = tracks_list_df['id_album'].unique().tolist()
    for i in range(0, len(albums_ids), 20):
        albums_ids_str = ','.join(albums_ids[i:i+20] )
        albums_ids_str = '?ids=' + albums_ids_str
        df_albums_aux = get_album(albums_ids_str)
        df_albums = pd.concat([df_albums, df_albums_aux], axis=0)

    # Filtra as colunas
    df_albums = df_albums[['id', 'name', 'popularity', 'release_date', 'total_tracks', 'type']]

    # Renomeia as colunas
    df_albums = df_albums.rename(columns={'id': 'id_album', 'name': 'album_name', 'popularity': 'album_popularity', 'release_date': 'album_release_date', 'total_tracks': 'album_total_tracks', 'type': 'album_type'})

    # Junta os dados das músicas com os dados dos albums
    tracks_list_df = tracks_list_df.merge(df_albums, on='id_album', how='left')
    

    # 4. Pega os dados do artista
    df_artists = pd.DataFrame()
    artists_ids = tracks_list_df['id_artist'].unique().tolist()
    for i in range(0, len(artists_ids), 50):
        artists_ids_str = ','.join(artists_ids[i:i+50] )
        artists_ids_str = '?ids=' + artists_ids_str
        df_artists_aux = get_artist_data(artists_ids_str)
        df_artists = pd.concat([df_artists, df_artists_aux], axis=0)

    # Filtra as colunas
    df_artists = df_artists[['id', 'name', 'genres', 'popularity', 'type', 'followers']]

    # Renomeia as colunas
    df_artists = df_artists.rename(columns={'id': 'id_artist', 'name': 'artist_name', 'genres': 'artist_genres', 'popularity': 'artist_popularity', 'type': 'artist_type', 'followers': 'artist_followers'})

    # Junta os dados das músicas com os dados dos artistas
    tracks_list_df = tracks_list_df.merge(df_artists, on='id_artist', how='left')

    # Pega o total de seguidores
    tracks_list_df['artist_followers'] = tracks_list_df['artist_followers'].apply(lambda x: x['total'])

    # Expande os IDs dos artistas para linhas separadas
    tracks_list_df = tracks_list_df.explode('artist_genres')

    # Mantém apenas os gêneros relevantes
    relevant_artist_genres = ['pop', 'hip hop', 'r&b', 'rap', 'reggae', 'rock', 'punk', 'alternative']
    tracks_list_df = tracks_list_df[tracks_list_df['artist_genres'].isin(relevant_artist_genres)]

    tracks_list_df = tracks_list_df[[
        'id', 'id_album', 'id_artist', 'name', 'explicit', 'duration_ms', 'popularity', # Track data
        'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', # Audio features
        'album_name', 'album_popularity', 'album_release_date', 'album_total_tracks', 'album_type', # Album data
        'artist_name', 'artist_genres', 'artist_popularity', 'artist_type', 'artist_followers' # Artist data
        ]]

    return tracks_list_df


In [ ]:
# Para cada música (linha do df_music_final), pegar as features

tracks_ids = df_musics_final['track_uri'].unique().tolist()
len(tracks_ids)

In [ ]:
# Parte 1

tracks_ids_part1 = tracks_ids[:len(tracks_ids)//2]

df_musics_features_final = pd.DataFrame()
print(f'Existem {len(tracks_ids_part1)} músicas para serem processadas')

for i in range(0, len(tracks_ids_part1), 50):
    print(f'Processando músicas {i} a {i+50}')
    try:
        df_aux = get_music_features(tracks_ids_part1[i:i+50])
        df_musics_features_final = pd.concat([df_musics_features_final, df_aux], axis=0)
    except:
        print(f'Erro processando músicas {i} a {i+50}', file=open('../data/01 - Extracted Data/df_musics_features_1_error_log.txt', 'a'))
        continue

df_musics_features_final.to_csv('../data/01 - Extracted Data/df_musics_features_1.csv', index=False)

In [ ]:
# Parte 2

tracks_ids_part2 = tracks_ids[len(tracks_ids)//2:]

df_musics_features_final = pd.DataFrame()
print(f'Existem {len(tracks_ids_part2)} músicas para serem processadas')

for i in range(0, len(tracks_ids_part2), 50):
    print(f'Processando músicas {i} a {i+50}')
    try:
        df_aux = get_music_features(tracks_ids_part2[i:i+50])
        df_musics_features_final = pd.concat([df_musics_features_final, df_aux], axis=0)
    except Exception as e:
        print(f'Erro processando músicas {i} a {i+50}')
        print(e, file=open('../data/01 - Extracted Data/df_musics_features_2_error_log.txt', 'a'))
        continue

df_musics_features_final.to_csv('../data/01 - Extracted Data/df_musics_features_2.csv', index=False)

In [ ]:
# Concatenando resultados

df_musics_features_1 = pd.read_csv('../data/01 - Extracted Data/df_musics_features_1.csv')
df_musics_features_2 = pd.read_csv('../data/01 - Extracted Data/df_musics_features_2.csv')

df_musics_final = pd.concat([df_musics_features_1, df_musics_features_2], axis=0)
df_musics_final.to_csv('../data/01 - Extracted Data/df_musics_features_final.csv', index=False)

Após os dados serem extraídos e a base final ser gerada, foi necessário fazer o upload da base para um repositório no GitHub, utilizando o GitHub Large File System (LFS).